
# IFT3395 Competition 2 - Hyperparam + Augmentation + TTA (NumPy)

组合策略：系统化超参搜索 + 在线数据增强 + 测试时增强 (TTA)，旨在进一步提升 Kaggle 分数。


In [1]:

import csv
import math
import pickle
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np

DATA_DIR = Path('data')
SEED = 2026
rng = np.random.default_rng(SEED)
np.set_printoptions(precision=4, suppress=True)


In [2]:

def load_split(split: str) -> Dict[str, np.ndarray]:
    path = DATA_DIR / f"{split}_data.pkl"
    with path.open('rb') as f:
        return pickle.load(f)

train_data = load_split('train')
test_data = load_split('test')

images = train_data['images'].astype(np.float32)
labels = train_data['labels'].reshape(-1).astype(int)
test_images = test_data['images'].astype(np.float32)
num_classes = len(np.unique(labels))
print(f"Train: {images.shape}, Test: {test_images.shape}, Classes: {num_classes}")


Train: (1080, 28, 28, 3), Test: (400, 28, 28, 3), Classes: 5


In [3]:

unique, counts = np.unique(labels, return_counts=True)
class_weights = counts.sum() / (counts.astype(np.float32) + 1e-6)
class_weights = class_weights / class_weights.mean()
print('Class weights:', class_weights)


Class weights: [0.2937 1.1151 0.6929 0.7357 2.1626]


In [4]:

def flatten_and_normalize(imgs: np.ndarray) -> np.ndarray:
    flat = imgs.reshape(len(imgs), -1).astype(np.float32)
    return flat / 255.0

class StandardScaler:
    def __init__(self):
        self.mean_: Optional[np.ndarray] = None
        self.std_: Optional[np.ndarray] = None

    def fit(self, x: np.ndarray) -> 'StandardScaler':
        self.mean_ = x.mean(axis=0, keepdims=True)
        self.std_ = x.std(axis=0, keepdims=True) + 1e-5
        return self

    def transform(self, x: np.ndarray) -> np.ndarray:
        if self.mean_ is None or self.std_ is None:
            raise ValueError('Scaler not fitted')
        return (x - self.mean_) / self.std_

    def fit_transform(self, x: np.ndarray) -> np.ndarray:
        return self.fit(x).transform(x)

x_norm = flatten_and_normalize(images)
x_test_norm = flatten_and_normalize(test_images)
scaler = StandardScaler().fit(x_norm)
x_std = scaler.transform(x_norm)
x_test_std = scaler.transform(x_test_norm)
SCALE_MEAN = scaler.mean_
SCALE_STD = scaler.std_


def standardize_raw(raw_batch: np.ndarray) -> np.ndarray:
    return (raw_batch - SCALE_MEAN) / SCALE_STD


def train_val_split_indices(n: int, val_ratio: float, seed: int) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    idx = rng.permutation(n)
    val_size = int(n * val_ratio)
    return idx[val_size:], idx[:val_size]


In [5]:

def augment_batch(raw_batch: np.ndarray, p_flip: float = 0.5, noise_std: float = 0.02, brightness: float = 0.08) -> np.ndarray:
    imgs = raw_batch.reshape(len(raw_batch), 28, 28, 3).copy()
    if np.random.rand() < p_flip:
        imgs = imgs[:, :, ::-1, :]
    if np.random.rand() < p_flip * 0.5:
        imgs = imgs[:, ::-1, :, :]
    imgs += np.random.normal(0.0, noise_std, size=imgs.shape)
    imgs += (np.random.rand() - 0.5) * brightness
    imgs = np.clip(imgs, 0.0, 1.0)
    return imgs.reshape(len(raw_batch), -1)

TTA_MODES = ['identity', 'hflip', 'vflip', 'bright+', 'bright-']


def apply_tta(raw_batch: np.ndarray, mode: str) -> np.ndarray:
    imgs = raw_batch.reshape(len(raw_batch), 28, 28, 3)
    if mode == 'hflip':
        aug = imgs[:, :, ::-1, :]
    elif mode == 'vflip':
        aug = imgs[:, ::-1, :, :]
    elif mode == 'bright+':
        aug = np.clip(imgs + 0.05, 0.0, 1.0)
    elif mode == 'bright-':
        aug = np.clip(imgs - 0.05, 0.0, 1.0)
    else:
        aug = imgs
    return aug.reshape(len(raw_batch), -1)


In [6]:

class MLP:
    def __init__(self, input_dim: int, num_classes: int, hidden_dim: int, lr: float, l2: float, dropout: float, seed: int = 0):
        self.hidden_dim = hidden_dim
        self.lr = lr
        self.base_lr = lr
        self.l2 = l2
        self.dropout = dropout
        rng = np.random.default_rng(seed)
        self.w1 = rng.normal(0.0, 0.02, size=(input_dim, hidden_dim))
        self.b1 = np.zeros(hidden_dim, dtype=np.float32)
        self.w2 = rng.normal(0.0, 0.02, size=(hidden_dim, num_classes))
        self.b2 = np.zeros(num_classes, dtype=np.float32)

    def _softmax(self, logits: np.ndarray) -> np.ndarray:
        logits = logits - logits.max(axis=1, keepdims=True)
        exp = np.exp(logits)
        return exp / exp.sum(axis=1, keepdims=True)

    def _forward(self, x: np.ndarray, train: bool = False) -> Tuple[np.ndarray, Tuple[np.ndarray, np.ndarray, np.ndarray]]:
        if train and self.dropout > 0.0:
            mask = (np.random.random(size=x.shape) >= self.dropout).astype(np.float32)
            x = x * mask
        z1 = x @ self.w1 + self.b1
        h1 = np.maximum(0.0, z1)
        logits = h1 @ self.w2 + self.b2
        return logits, (x, z1, h1)

    def _loss_and_grads(self, x: np.ndarray, y: np.ndarray, class_weights: np.ndarray) -> Tuple[float, Dict[str, np.ndarray]]:
        logits, cache = self._forward(x, train=True)
        probs = self._softmax(logits)
        batch = x.shape[0]
        sample_w = class_weights[y]
        loss = -np.sum(sample_w * np.log(probs[np.arange(batch), y] + 1e-8)) / sample_w.sum()
        loss += 0.5 * self.l2 * (np.sum(self.w1 * self.w1) + np.sum(self.w2 * self.w2))
        grad_logits = probs
        grad_logits[np.arange(batch), y] -= 1.0
        grad_logits *= (sample_w[:, None] / sample_w.sum())
        x_cache, z1, h1 = cache
        grads = {}
        grads['w2'] = h1.T @ grad_logits + self.l2 * self.w2
        grads['b2'] = grad_logits.sum(axis=0)
        grad_hidden = grad_logits @ self.w2.T
        grad_hidden[z1 <= 0.0] = 0.0
        grads['w1'] = x_cache.T @ grad_hidden + self.l2 * self.w1
        grads['b1'] = grad_hidden.sum(axis=0)
        return loss, grads

    def fit(self, x: np.ndarray, y: np.ndarray, *, class_weights: np.ndarray, epochs: int, batch_size: int, val_data: Optional[Tuple[np.ndarray, np.ndarray]] = None, raw_data: Optional[np.ndarray] = None, augment: bool = False) -> Dict[str, List[float]]:
        history = {'train_acc': [], 'val_acc': []}
        num_samples = x.shape[0]
        for epoch in range(epochs):
            lr_scale = 0.5 * (1 + math.cos(math.pi * epoch / epochs))
            self.lr = self.base_lr * lr_scale
            indices = np.random.permutation(num_samples)
            for start in range(0, num_samples, batch_size):
                end = start + batch_size
                idx = indices[start:end]
                xb = x[idx]
                if augment and raw_data is not None:
                    raw_aug = augment_batch(raw_data[idx])
                    xb = standardize_raw(raw_aug)
                yb = y[idx]
                _, grads = self._loss_and_grads(xb, yb, class_weights)
                self.w1 -= self.lr * grads['w1']
                self.b1 -= self.lr * grads['b1']
                self.w2 -= self.lr * grads['w2']
                self.b2 -= self.lr * grads['b2']
            history['train_acc'].append(self.accuracy(x, y))
            if val_data is not None:
                history['val_acc'].append(self.accuracy(*val_data))
        return history

    def predict_proba(self, x: np.ndarray) -> np.ndarray:
        logits, _ = self._forward(x, train=False)
        return self._softmax(logits)

    def predict(self, x: np.ndarray) -> np.ndarray:
        return np.argmax(self.predict_proba(x), axis=1)

    def accuracy(self, x: np.ndarray, y: np.ndarray) -> float:
        return float((self.predict(x) == y).mean())


In [7]:

train_idx, val_idx = train_val_split_indices(len(x_std), val_ratio=0.2, seed=SEED)
x_train_std = x_std[train_idx]
x_val_std = x_std[val_idx]
x_train_raw = x_norm[train_idx]
x_val_raw = x_norm[val_idx]
y_train = labels[train_idx]
y_val = labels[val_idx]
print(f"Train subset: {x_train_std.shape}, Val subset: {x_val_std.shape}")


Train subset: (864, 2352), Val subset: (216, 2352)


In [8]:

search_space = [
    {'hidden_dim': 384, 'lr': 0.055, 'l2': 5e-5, 'dropout': 0.08, 'epochs': 260, 'batch_size': 128},
    {'hidden_dim': 512, 'lr': 0.06, 'l2': 8e-5, 'dropout': 0.12, 'epochs': 300, 'batch_size': 112},
    {'hidden_dim': 640, 'lr': 0.07, 'l2': 1e-4, 'dropout': 0.15, 'epochs': 320, 'batch_size': 96},
    {'hidden_dim': 768, 'lr': 0.05, 'l2': 6e-5, 'dropout': 0.1, 'epochs': 340, 'batch_size': 128},
    {'hidden_dim': 512, 'lr': 0.045, 'l2': 3e-5, 'dropout': 0.05, 'epochs': 280, 'batch_size': 96},
]

results = []
for i, params in enumerate(search_space, 1):
    print(f"Config {i}/{len(search_space)}: {params}")
    model = MLP(
        input_dim=x_train_std.shape[1],
        num_classes=num_classes,
        hidden_dim=params['hidden_dim'],
        lr=params['lr'],
        l2=params['l2'],
        dropout=params['dropout'],
        seed=SEED + i
    )
    history = model.fit(
        x_train_std,
        y_train,
        class_weights=class_weights,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        val_data=(x_val_std, y_val),
        raw_data=x_train_raw,
        augment=True
    )
    train_acc = history['train_acc'][-1]
    val_acc = history['val_acc'][-1]
    results.append({**params, 'train_acc': train_acc, 'val_acc': val_acc})
    print(f"  train acc={train_acc:.4f}, val acc={val_acc:.4f}\n")

best = max(results, key=lambda r: r['val_acc'])
print('Best config:', {k: v for k, v in best.items() if k not in {'train_acc', 'val_acc'}})


Config 1/5: {'hidden_dim': 384, 'lr': 0.055, 'l2': 5e-05, 'dropout': 0.08, 'epochs': 260, 'batch_size': 128}
  train acc=0.5961, val acc=0.4861

Config 2/5: {'hidden_dim': 512, 'lr': 0.06, 'l2': 8e-05, 'dropout': 0.12, 'epochs': 300, 'batch_size': 112}
  train acc=0.5035, val acc=0.3704

Config 3/5: {'hidden_dim': 640, 'lr': 0.07, 'l2': 0.0001, 'dropout': 0.15, 'epochs': 320, 'batch_size': 96}
  train acc=0.3576, val acc=0.1759

Config 4/5: {'hidden_dim': 768, 'lr': 0.05, 'l2': 6e-05, 'dropout': 0.1, 'epochs': 340, 'batch_size': 128}
  train acc=0.2940, val acc=0.1481

Config 5/5: {'hidden_dim': 512, 'lr': 0.045, 'l2': 3e-05, 'dropout': 0.05, 'epochs': 280, 'batch_size': 96}
  train acc=0.5949, val acc=0.4676

Best config: {'hidden_dim': 384, 'lr': 0.055, 'l2': 5e-05, 'dropout': 0.08, 'epochs': 260, 'batch_size': 128}


In [9]:

TOP_K = 2
sorted_configs = sorted(results, key=lambda r: r['val_acc'], reverse=True)[:TOP_K]
models: List[MLP] = []
for idx, cfg in enumerate(sorted_configs):
    cfg_full = cfg.copy()
    cfg_full['epochs'] = cfg['epochs'] + 40
    print(f"Retraining top config {idx + 1}: {cfg_full}")
    model = MLP(
        input_dim=x_std.shape[1],
        num_classes=num_classes,
        hidden_dim=cfg_full['hidden_dim'],
        lr=cfg_full['lr'],
        l2=cfg_full['l2'],
        dropout=cfg_full['dropout'],
        seed=SEED * 2 + idx
    )
    model.fit(
        x_std,
        labels,
        class_weights=class_weights,
        epochs=cfg_full['epochs'],
        batch_size=cfg_full['batch_size'],
        val_data=None,
        raw_data=x_norm,
        augment=True
    )
    models.append(model)
print(f"Total models trained: {len(models)}")


Retraining top config 1: {'hidden_dim': 384, 'lr': 0.055, 'l2': 5e-05, 'dropout': 0.08, 'epochs': 300, 'batch_size': 128, 'train_acc': 0.5960648148148148, 'val_acc': 0.4861111111111111}
Retraining top config 2: {'hidden_dim': 512, 'lr': 0.045, 'l2': 3e-05, 'dropout': 0.05, 'epochs': 320, 'batch_size': 96, 'train_acc': 0.5949074074074074, 'val_acc': 0.4675925925925926}
Total models trained: 2


In [10]:

probs_accum = np.zeros((len(x_test_std), num_classes), dtype=np.float32)
for model in models:
    for mode in TTA_MODES:
        raw_aug = apply_tta(x_test_norm, mode)
        std_aug = standardize_raw(raw_aug)
        probs_accum += model.predict_proba(std_aug)

probs_accum /= (len(models) * len(TTA_MODES))
test_preds = np.argmax(probs_accum, axis=1)

ids = [str(i) for i in range(1, len(test_preds) + 1)]
submission_path = Path('submission_aug_tta.csv')
with submission_path.open('w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['ID', 'Label'])
    for idx, label in zip(ids, test_preds):
        writer.writerow([idx, int(label)])

print(f'Submission saved to {submission_path.resolve()}')


Submission saved to C:\Users\yudim\Downloads\IFT3395_Competition2\submission_aug_tta.csv
